## An implementation of sequence to sequence learning for performing addition
```
Input: "535+61"
Output: "596"
```

Padding is handled by using a repeated sentinel character (space). Input may optionally be inverted, shown to increase performance in many tasks in:
- [Learning to Execute](http://arxiv.org/abs/1410.4615)
- [Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)

Theoretically it introduces shorter term dependencies between source and target.
Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

In [1]:
from __future__ import print_function
from keras.models import Sequential, slice_X
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.layers import recurrent
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
class CharacterTable(object):
    '''
    Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    '''
    def __init__(self, chars, maxlen):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c,i) for i, c in enumerate(self.chars))
        self.indices_char = dict([reversed(i) for i in self.char_indices.items()])
        
    def encode(self, C, maxlen=None):
        '''Encode a set o characters them to a one hot integer representation'''
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.chars)))
        for i,c in enumerate(C):
            X[i, self.char_indices[c]] = 1
        return X
    
    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in X)
    

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True
# Try replacing GRU, or SimpleRNN
RNN = recurrent.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 2
MAXLEN = DIGITS + 1 + DIGITS

chars = '0123456789+ '
ctable = CharacterTable(chars, MAXLEN)

In [5]:
questions = []
expected = []
seen = set()

while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that X+Y == Y+X (hence the sorting)
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Total addition questions: 50000


In [7]:
print('Vectorization...')
X = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.int)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.int)
for i, sentence in enumerate(questions):
    X[i] = ctable.encode(sentence, maxlen=MAXLEN)
# encode expcted results from 3 char digit + space to (4,12) tensor
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, maxlen=DIGITS + 1)

Vectorization...


In [8]:
print('The shape of X is: %s, while the shape of X[0] is %s' % (X.shape, X[1].shape))
print('The shape of Y is: %s, while the shape of y[0] is %s' % (y.shape, y[1].shape))

The shape of X is: (50000, 7, 12), while the shape of X[0] is (7, 12)
The shape of Y is: (50000, 4, 12), while the shape of y[0] is (4, 12)


In [ ]:
# Shuffle (X, y) in unison as the later parts of X will almost all be larger digits
indices = np.arange(len(y))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

In [ ]:
# Explicitly set apart 10% for validation data that we never train over
split_at = len(X) - len(X) / 10
(X_train, X_val) = (slice_X(X, 0, split_at), slice_X(X, split_at))
(y_train, y_val) = (y[:split_at], y[split_at:])

print(X_train.shape)
print(y_train.shape)

In [ ]:
print("Build model...")
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributedDense(len(chars)))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X_train,y_train, batch_size=BATCH_SIZE, nb_epoch=1,
             validation_data=(X_val, y_val), show_accuracy=True)
    
    ###
    # Select 10 samples from the validation set at random so we can visualize errors
    for i in range(10):
        ind = np.random.randint(0, len(X_val))
        rowX, rowy = X_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowX, verbose=0)
        q = ctable.decode(rowX[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        print(colors.ok + '☑' + colors.close if correct == guess else colors.fail + '☒' + colors.close, guess)
        print('---')

In [ ]:
ind = np.random.randint(0, len(X_val))

In [ ]:
rowX, rowy = X_val[np.array([ind])], y_val[np.array([ind])]

In [ ]:
preds = model.predict_classes(rowX, verbose=0)
probs = model.predict_proba(rowX, verbose=0)

In [ ]:
q = ctable.decode(rowX[0]) 
correct = ctable.decode(rowy[0])

In [ ]:
guess = ctable.decode(preds[0], calc_argmax=False)

In [ ]:
print('Q', q[::-1] if INVERT else q)
print('T', correct)
print(colors.ok + '☑' + colors.close if correct == guess else colors.fail + '☒' + colors.close, guess)

In [ ]:
guess

In [ ]:
preds

In [ ]:
X = preds[0]
''.join(ctable.indices_char[x] for x in X)

In [ ]:
print(X)

In [ ]:
ctable.indices_char

In [ ]:
preds